In [ ]:
import os
import re

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
pd.options.plotting.backend = "matplotlib"
# pd.options.plotting.backend = "plotly"
template = "plotly_dark"

In [ ]:
def load_price_timeseries(file: str) -> pd.Series:
    df = pd.read_csv(file)
    df.index = pd.to_datetime(df["Date"], format="%d/%m/%Y %H:%M")
    return df["Intraday Continuous 15 minutes ID1-Price"]  # * 1e-6 # €/MWh -> €/Wh

In [ ]:
def load_imbalance_prices(file: str) -> pd.Series:
    df = pd.read_csv(file, sep=";", decimal=",")
    t = pd.to_datetime(df["Datum"] + " " + df["von"], format='%d.%m.%Y %H:%M') 
    df["datetime"] = pd.date_range(t.iloc[0], t.iloc[-1], freq="15Min") # avoid repeated indices from daylight saving time change
    df = df.set_index("datetime")
    return df

In [ ]:
def load_results(dir):
    res = {}
    files = [f for f in os.listdir(dir) if f.endswith(".parquet")]
    for file in files:
        name, _ = os.path.splitext(file)
        res[name] = pd.read_parquet(dir + file)

    return res

In [ ]:
def exctract_value(string, key):
    match = re.search(fr"{key}=([\d.]+)", string)
    if match:
        value = match.group(1)
    return float(value)

In [ ]:
def calc_fec(df):
    power = df["power_sim"]
    power_pos = power[power > 0].sum() * (1 / 60)
    power_neg = power[power < 0].abs().sum() * (1 / 60)
    return (power_pos + power_neg) / 2 / 180e3

In [ ]:
def calc_roundtrip_efficiency(res):
    p = res["power_sim"]
    e_pos = p[p > 0].abs().sum() * (1 / 60)  # Wh
    e_neg = p[p < 0].abs().sum() * (1 / 60)  # Wh

    delta_soc = res["soc_sim"].iloc[-1] - res["soc_sim"].iloc[0]
    delta_e = delta_soc * 180e3  # Wh

    return abs(e_neg) / (e_pos - delta_e)

In [ ]:
def calc_revenue(df, price):
    price = price.resample("1Min").ffill()
    df = df.join(price)
    return -1 * sum(df["power_sim"] * df["Intraday Continuous 15 minutes ID1-Price"]) * (1/60) * 1e-6  # W -> MWh

In [ ]:
def calculate_imbalance_cost(df, price):
    price = price.resample("1Min").ffill()
    df = df.join(price)
    df["imb"] = -(df["power_opt"] - df["power_sim"]) * (1 / 60) * 1e-6 # MWh
    # negation since positive power is charging
    
    # BESS under-supply
    df_u = df.loc[df.imb > 0]
    cost_u = sum(df_u["imb"] * df_u["reBAP unterdeckt"])
    
    # BESS over-supply
    df_o = df.loc[df.imb < 0]
    cost_o = sum(df_o["imb"] * df_o["reBAP ueberdeckt"])

    return cost_u + cost_o

In [ ]:
def calc_imbalance_pos(df):
    df["imb"] = -(df["power_opt"] - df["power_sim"]) * (1 / 60) #* 1e-6 # MWh
    # negation since positive power is charging
    
    # BESS under-supply
    return df.loc[df.imb > 0, "imb"].sum()

In [ ]:
def calc_imbalance_neg(df):
    df["imb"] = -(df["power_opt"] - df["power_sim"]) * (1 / 60) #* 1e-6 # MWh
    # negation since positive power is charging
    
    # BESS over-supply
    return df.loc[df.imb < 0, "imb"].sum()

In [ ]:
def analyse_results_lp(res, price, price_imb):
    out = pd.DataFrame()
    for (id, df) in res.items():
        if "LP" in id:
            data = dict(
                dt = exctract_value(id, "dt"),
                fec_limit = exctract_value(id, "fec"),
                r = exctract_value(id, "r"),
                eff = exctract_value(id, "eff"),
                fec = calc_fec(df),
                rte = calc_roundtrip_efficiency(df),
                rev = calc_revenue(df, price),
                imb_under = calc_imbalance_pos(df),
                imb_over = calc_imbalance_neg(df),
            )
            out = pd.concat([out, pd.DataFrame(data=[data])])

    # out["total"] = out["rev"] - out["imb"]
    out["imb_total"] = out["imb_under"] - out["imb_over"]

    return out

In [ ]:
def analyse_results_nl(res, price, price_imb):
    out = pd.DataFrame()
    for (id, df) in res.items():
        if ("NL" in id) and ("min" not in id):
        # if ("NL" in id) and ("300" in id):
            data = dict(
                dt = exctract_value(id, "dt"),
                fec_limit = exctract_value(id, "fec"),
                r = exctract_value(id, "r"),
                r_opt = exctract_value(id, "r_opt"),
                fec = calc_fec(df),
                rte = calc_roundtrip_efficiency(df),
                rev = calc_revenue(df, price),
                imb_under = calc_imbalance_pos(df),
                imb_over = calc_imbalance_neg(df),
            )
            out = pd.concat([out, pd.DataFrame(data=[data])])

    # out["total"] = out["rev"] - out["imb"]
    out["imb_total"] = out["imb_under"] - out["imb_over"]

    return out

In [ ]:
def plot_timeseries(df, **kwargs):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
    fig.update_layout(**kwargs)

    for id in ["soc_sim", "soc_opt"]:
        fig.add_trace(go.Scatter(x=df.index, y=df[id], name=id), row=1, col=1)

    for id in ["power_sim", "power_opt"]:
        fig.add_trace(go.Scatter(x=df.index, y=df[id], name=id), row=2, col=1)
    
    return fig

In [ ]:
def plot_trading_power(res, price):
    fig, ax = plt.subplots(2, 2, height_ratios=[1, 3], width_ratios=[3, 1])

    price = price.resample("1Min").ffill()
    df = res.join(price)
    power = df["power_sim"] / 100e3
    price = df["Intraday Continuous 15 minutes ID1-Price"]

    idx = power > 0
    # ax.scatter(price.loc[idx], power.loc[idx])
    ax[1, 0].scatter(power.loc[idx], price.loc[idx], alpha=0.5)
    ax[0, 0].hist(power.loc[idx], alpha=0.5, label="Charge")
    ax[1, 1].hist(price.loc[idx], orientation="horizontal", alpha=0.5)

    idx = power < 0
    # ax.scatter(price.loc[idx], power.loc[idx])
    ax[1, 0].scatter(power.loc[idx], price.loc[idx], alpha=0.5)
    ax[0, 0].hist(power.loc[idx], alpha=0.5, label="Discharge")
    ax[1, 1].hist(price.loc[idx], orientation="horizontal", alpha=0.5)

    ax[0, 1].set_visible(False)
    ax[0, 0].xaxis.set_visible(False)
    ax[1, 1].yaxis.set_visible(False)

    # ax[0,0].set_ylabel("Hist")
    # ax[1,1].set_xlabel("Hist")
    ax[1, 0].set_xlabel("Power in p.u.")
    ax[1, 0].set_ylabel("Price in €/MWh")

    fig.tight_layout(h_pad=0.1, w_pad=0.5)

    fig.legend(loc="center", bbox_to_anchor=(0.87, 0.85))

    return fig

In [ ]:
def plot_eff_rev(df_lp, df_nl):
    fig, ax = plt.subplots()
    max_rev = 1 #max(df_lp["rev"].max(), df_nl["rev"].max())
    ax.scatter(df_lp["rte"], df_lp["rev"] / max_rev, label="LP")
    ax.scatter(df_nl["rte"], df_nl["rev"] / max_rev, label="NL")
    ax.legend()
    return fig

In [ ]:
def plot_imb(df_lp, df_nl):
    fig, ax = plt.subplots()

    r_values = df_lp['r'].unique()
    x = np.arange(len(r_values))
    bar_width = 0.35

    ax.bar(x - bar_width/2, df_lp['imb_under'] * 1e-3, width=bar_width, label='LP - over')
    ax.bar(x - bar_width/2, df_lp['imb_over'] * 1e-3, width=bar_width, label='LP - under')
    ax.bar(x + bar_width/2, df_nl['imb_under'] * 1e-3, width=bar_width, label='NL - over')
    ax.bar(x + bar_width/2, df_nl['imb_over'] * 1e-3, width=bar_width, label='NL - under')
#
    # ax.set_ylim(0.85, 1.02)
    ax.set_xticks(x)
    ax.set_xticklabels(r_values)
    ax.set_xlabel("SOH-R")
    ax.set_ylabel("Imbalance energy in kWh")
    ax.legend(title="Model")

    return fig

In [ ]:
price_2019 = load_price_timeseries("data/intraday_prices/electricity_prices_germany_2019.csv")
price_2021 = load_price_timeseries("data/intraday_prices/electricity_prices_germany_2021.csv")
price_2022 = load_price_timeseries("data/intraday_prices/electricity_prices_germany_2022.csv")

In [ ]:
imb_2021 = load_imbalance_prices("data/balancing_energy_prices/reBAP_2021.csv")
imb_2022 = load_imbalance_prices("data/balancing_energy_prices/reBAP_2022.csv")

In [ ]:
dir = "results/dae-3/"
res = load_results(dir)

res_2021 = {key: value for key, value in res.items() if "2021" in key}
# res_2021 = {key: value for key, value in res.items() if "dt=60" in key}

df_lp_2021 = analyse_results_lp(res_2021, price_2021, imb_2021)
df_nl_2021 = analyse_results_nl(res_2021, price_2021, imb_2021)

In [ ]:
fig = plot_eff_rev(df_lp_2021, df_nl_2021)

In [ ]:
fig = plot_eff_rev(df_lp_2021[df_lp_2021.eff == 0.95], df_nl_2021[df_nl_2021.r_opt == 1.0])

In [ ]:
df_nl_2021[(df_nl_2021.r == 1.0) & (df_nl_2021.dt == 180)].plot.scatter(x="r_opt", y="rev")

In [ ]:
def plot_imb_power(df_lp, df_nl):
    # df_lp = res["2021 LP fec=1.5 r=1.0 eff=0.95"]
    df_lp["imb"] = -(df_lp["power_opt"] - df_lp["power_sim"])

    # df_nl = res["2021 NL fec=1.5 r=1.0 r_opt=1.0 dt=900"]
    df_nl["imb"] = -(df_nl["power_opt"] - df_nl["power_sim"])

    fig, ax = plt.subplots()
    ax.plot(df_lp.index, df_lp["imb"] / 180e3, label="LP", alpha=0.8)
    ax.plot(df_lp.index, df_nl["imb"] / 180e3, label="NL", alpha=0.8)
    ax.legend(loc="upper right", title="Model")
    ax.set_ylabel("Imbalance power in p.u.")

    return fig

In [ ]:
def plot_diff(price, *dfs, **kwargs):
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True)
    fig.update_layout(**kwargs)

    price = price.resample("1min").ffill()
    fig.add_trace(go.Scatter(x=price.index, y=price.values, line_shape="hv", name="price"), row=1, col=1)

    for (i, df) in enumerate(dfs):
        fig.add_trace(go.Scatter(x=df.index, y=df["power_opt"], name=f"power {i}"), row=2, col=1)    

    for (i, df) in enumerate(dfs):
        fig.add_trace(go.Scatter(x=df.index, y=df["soc_opt"], name=f"soc {i}"), row=3, col=1)

    for (i, df) in enumerate(dfs):
        df = df.join(price)
        rev = -(df["power_sim"] * df["Intraday Continuous 15 minutes ID1-Price"]).cumsum() * (1/60) * 1e-6  # W -> MWh
        fig.add_trace(go.Scatter(x=df.index, y=rev, name=f"rev {i}"), row=4, col=1)

    return fig

In [ ]:
plot_diff(
    price_2021.loc["2021-01-01":"2021-01-07"],
# res_2021["2021 LP fec=1.5 r=3.0 eff=0.94 dt=300"].loc["2021-01-01":"2021-01-07"],
res_2021["2021 LP fec=1.5 r=3.0 eff=0.94 dt=180"].loc["2021-01-01":"2021-01-07"],
res_2021["2021 LP fec=1.5 r=3.0 eff=0.94 dt=60"].loc["2021-01-01":"2021-01-07"],
# res_2021["2021 NL fec=1.5 r=3.0 r_opt=1.0 dt=300"].loc["2021-01-01":"2021-01-07"],
# res_2021["2021 NL fec=1.5 r=3.0 r_opt=1.0 dt=180"].loc["2021-01-01":"2021-01-07"],
# res_2021["2021 NL fec=1.5 r=3.0 r_opt=1.0 dt=60"].loc["2021-01-01":"2021-01-07"],
template=template,
height=1200
)